<a href="https://colab.research.google.com/github/bluesnake1021/whisper-ui-colab/blob/main/Whisper_YouTube_in_SRT_ipynb_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title INSTALL DEPENDENCIES
!pip install -q ffmpeg-python git+https://github.com/openai/whisper.git > /dev/null --q
!pip install -q gradio
!pip install -q pytube
!pip install -q vtt_to_srt3

In [ ]:
#@title Launch the APP

import gradio as gr
from pytube import YouTube
import os
import subprocess
import uuid
from vtt_to_srt import vtt_to_srt

def transcribe_youtube(source_language, model_size, youtube_link, source_file=None):
  audio = str(uuid.uuid4()) + '.m4a'
  if(os.path.exists(audio)):
    os.remove(audio)
    os.remove(audio + '.vtt')
    os.remove(audio + '.srt')

  if youtube_link != "":
    yt = YouTube(youtube_link)
    yt.streams.filter(only_audio=True)[0].download(filename=audio)
    subprocess.run('whisper ' + audio + ' --language ' + str(source_language) + ' --model ' + str(model_size), shell=True)
  
  # A logic to process an uploaded audio file
  elif source_file is not None:
    subprocess.run('whisper ' + source_file + ' --language ' + str(source_language) + ' --model ' + str(model_size), shell=True)
    audio = source_file.split("/")[2]
  else:
      return "You must either provide a YouTube link or a source file"
  
  path = '/content/' + audio + '.vtt'
  vtt_to_srt(path)
  with open('/content/' + audio + '.srt','r') as file:
    data=file.read()
  return data,'/content/' + audio + '.srt'

demo = gr.Interface(
    fn=transcribe_youtube,
    inputs=[
        gr.Textbox(lines=1, placeholder="enter the source language (Japanese, English, etc)"),
        gr.Dropdown(label="Model Size", choices=['base','small', 'medium', 'large'], value='large'),
        gr.Textbox(lines=2, placeholder="paste youtube link here"),
        # A gradio inteface to upload an audio file
        gr.Audio(source="upload", type="filepath")
    ],
    outputs=[gr.Textbox(label="Transcripts"),gr.File(label="Download Transcripts")],
    allow_flagging="never"
)
demo.launch(debug=True)

## Transcribe an audio file that is uploaded to the content folder.

In [ ]:
# Change the "audio.m4a" file name so that it matches with the file name
# you uploaded to the content folder

!whisper audio.m4a --language Japanese --model large

## Translate transcripts into English

In [ ]:
# Again change the "audio.m4a" file name so that it matches with the file name
# you uploaded to the content folder

!whisper audio.m4a --language Japanese --model large --task translate